In [1]:
%matplotlib inline
from data_tools import *
from algorithms import *
from plot_lib import *
import matplotlib.pyplot as plt
# from nets import *
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split
from sklearn.feature_selection import RFE
from sklearn import preprocessing
import numpy as np
import code 
import caffe
import scipy as nd
from caffe import layers as L
from caffe import params as P
import lmdb

In [2]:
print "Splitting the data into train and test set."

Splitting the data into train and test set.


In [23]:
print "Create Net-Architecture."
def cnn(lmdb, batch_size):
    n = caffe.NetSpec()
    n.data, n.label = L.Data(batch_size=batch_size, backend=P.Data.LMDB, source=lmdb, transform_param=dict(scale=1./16293), ntop=2)
    
    n.conv1 = L.Convolution(n.data, kernel_size=5, num_output=20, weight_filler=dict(type='xavier'))
    n.pool1 = L.Pooling(n.conv1, kernel_size=2, stride=2, pool=P.Pooling.MAX)
    
    n.conv2 = L.Convolution(n.pool1, kernel_size=5, num_output=50, weight_filler=dict(type='xavier'))
    n.pool2 = L.Pooling(n.conv2, kernel_size=2, stride=2, pool=P.Pooling.MAX)
    
    n.ip1 = L.InnerProduct(n.pool2, num_output=500, weight_filler=dict(type='xavier'))
    n.relu1 = L.ReLU(n.ip1, in_place=True)
    
    n.ip2 = L.InnerProduct(n.relu1, num_output=10, weight_filler=dict(type='xavier'))
    n.loss = L.SoftmaxWithLoss(n.ip2, n.label)
    return n.to_proto()

def cnn2(lmdb, batch_size):
    n = caffe.NetSpec()
    n.data, n.label = L.Data(batch_size=batch_size, backend=P.Data.LMDB, source=lmdb, transform_param=dict(scale=1./16293), ntop=2)
    
    n.conv1 = L.Convolution(n.data, kernel_size=4, num_output=32, weight_filler=dict(type='xavier'))
    n.relu1 = L.ReLU(n.conv1, in_place=True)
    n.drop1 = L.Dropout(n.relu1,dropout_ratio=0.2)
    n.pool1 = L.Pooling(n.drop1, kernel_size=2, stride=2, pool=P.Pooling.MAX)
    
    n.conv2 = L.Convolution(n.pool1, kernel_size=4, num_output=64, weight_filler=dict(type='xavier'))
    n.relu2 = L.ReLU(n.conv2, in_place=True)
    n.pool2 = L.Pooling(n.relu2, kernel_size=2, stride=2, pool=P.Pooling.MAX)
    n.drop2 = L.Dropout(n.pool2, dropout_ratio=0.2)
    
    n.conv3 = L.Convolution(n.drop2, kernel_size=3, num_output=64, weight_filler=dict(type='xavier'))
    n.relu3 = L.ReLU(n.conv3, in_place=True)
    n.pool3 = L.Pooling(n.relu3, kernel_size=2, stride=2, pool=P.Pooling.MAX)
    n.drop3 = L.Dropout(n.pool3, dropout_ratio=0.25)
    
    n.ip4 = L.InnerProduct(n.drop3, num_output=100, weight_filler=dict(type='xavier'))
    n.relu4 = L.ReLU(n.ip4, in_place=True)
    n.drop4 = L.Dropout(n.relu4,dropout_ratio=0.5)
    
    n.ip5 = L.InnerProduct(n.drop4, num_output=4, weight_filler=dict(type='xavier'))
    n.loss = L.SoftmaxWithLoss(n.ip5, n.label)    
    return n.to_proto()
    

print "Write train.prototxt"
with open('ModelZoo/cell_net_1/train.prototxt', 'w') as f:
    f.write(str(cnn2('/Volumes/MoritzBertholdHD/CellData/Experiments/Ex1/PreparedData/all_channels_80_80_full_no_zeros_in_cells', 64)))
print "Write test.prototxt"
with open('ModelZoo/cell_net_1/test.prototxt', 'w') as f:
    f.write(str(cnn2('/Volumes/MoritzBertholdHD/CellData/Experiments/Ex2/PreparedData/all_channels_80_80_full_no_zeros_in_cells', 100)))

Create Net-Architecture.
Write train.prototxt
Write test.prototxt


In [24]:
caffe.set_mode_cpu()

In [25]:
solver = None
solver = caffe.get_solver('ModelZoo/cell_net_1/solver.prototxt')

In [26]:
print"Layers' features:"
[(k, v.data.shape) for k, v in solver.net.blobs.items()]

Layers' features:


[('data', (64, 4, 80, 80)),
 ('label', (64,)),
 ('conv1', (64, 32, 77, 77)),
 ('drop1', (64, 32, 77, 77)),
 ('pool1', (64, 32, 39, 39)),
 ('conv2', (64, 64, 36, 36)),
 ('pool2', (64, 64, 18, 18)),
 ('drop2', (64, 64, 18, 18)),
 ('conv3', (64, 64, 16, 16)),
 ('pool3', (64, 64, 8, 8)),
 ('drop3', (64, 64, 8, 8)),
 ('ip4', (64, 100)),
 ('drop4', (64, 100)),
 ('ip5', (64, 4)),
 ('loss', ())]

In [27]:
print "Parameters and shape:"
[(k, v[0].data.shape) for k, v in solver.net.params.items()]

Parameters and shape:


[('conv1', (32, 4, 4, 4)),
 ('conv2', (64, 32, 4, 4)),
 ('conv3', (64, 64, 3, 3)),
 ('ip4', (100, 4096)),
 ('ip5', (4, 100))]

In [28]:
solver.net.forward()

{'loss': array(1.386287808418274, dtype=float32)}

In [29]:
solver.test_nets[0].forward()

{'loss': array(1.3844592571258545, dtype=float32)}

In [30]:
solver.net.backward()

{}

In [ ]:
solver.solve()
